In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt


from helpers.utils import creat_datasets, creating_val_data, reset_graph, grid_serach, read_pickle_file, write_pickle_file
from helpers.utils import creating_train_val_test_datasets, tensorboard_callback,  save_best_model_callback
from helpers.bln_layer import  bln_layer
from helpers.dense_layer import  dense_layer
from helpers.bln_callback import bln_callback 




from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.datasets import imdb

In [2]:
tf.config.experimental_run_functions_eagerly(True)

In [3]:
tf.__version__

'2.1.0'

### Loading data

In [5]:
random_seed = 100
minibatch = 25
buffersize = 60000
number_valid_sampels = 5000 # number of validation data
epochs = 10
learning_rate = 0.003 #0.005

#((25000  - 5000 )/ 25 ) * .2
number_batches_train = 160 # number of batches to train, each batch of size minibatch parameter
number_batches_valid = 40 # number of batches to validate, each batch of size minibatch parameter (( 5000 )/ 25 ) * .2
num_classes = 1

max_features = 20000
maxlen = 80  # cut texts after this number of words (among top max_features most common words)


In [6]:
print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen = maxlen)
x_test = sequence.pad_sequences(x_test, maxlen = maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

Loading data...


C:\Users\Rima\.conda\envs\bln\lib\site-packages\tensorflow_core\python\keras\datasets\imdb.py:129: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
C:\Users\Rima\.conda\envs\bln\lib\site-packages\tensorflow_core\python\keras\datasets\imdb.py:130: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_test, y_test = np.array(xs[idx:]), np.array(labels[idx:])


25000 train sequences
25000 test sequences
Pad sequences (samples x time)
x_train shape: (25000, 80)
x_test shape: (25000, 80)


In [7]:
x_train, y_train, x_valid, y_valid = creating_val_data(x_train, y_train,
                                                       number_valid_sampels = number_valid_sampels,
                                                       random_seed=random_seed)

train_dataset, valid_dataset, test_dataset = creating_train_val_test_datasets(x_train, y_train,
                                                                              x_test, y_test,
                                                                              x_valid, y_valid, 
                                                                              minibatch = minibatch,
                                                                              buffersize= buffersize,
                                                                              random_seed=random_seed)

In [8]:
train_dataset

<PrefetchDataset shapes: ((25, 80), (25,)), types: (tf.int32, tf.int64)>

# 1. Using Batch Layer Normalization Layer

In [9]:
def BLNLayer_model(inputshape= (80), max_features = 20000, embed_size = 512, random_seed = 100,
                          lstm_unit = 128 , dense_units= 1, batch_size = 10,
                          b_mm = True, b_mv = True, f_mm = False, f_mv = False):
    
   
    # building the model
  
    input_lyr = tf.keras.Input(shape = inputshape, batch_size=batch_size, name = 'input')
    
    x = Embedding(max_features, embed_size,
                  embeddings_initializer=tf.keras.initializers.GlorotUniform(seed=random_seed))(input_lyr)
    
    x = LSTM(lstm_unit, dropout=0.2, recurrent_dropout=0.2,activation='tanh',
            recurrent_initializer = tf.keras.initializers.GlorotUniform(seed=random_seed),
            kernel_initializer = tf.keras.initializers.GlorotUniform(seed=random_seed) ,
            return_sequences=True)(x)
    
    x = bln_layer(stateful = True, batchsize= batch_size, name = 'bn1', 
                  batch_moving_mean = b_mm, batch_moving_var = b_mv,
                  feature_moving_mean = f_mm, feature_moving_var = f_mv)(x) 
    
    x = LSTM(lstm_unit//2, dropout=0.2,  recurrent_dropout=0.2,activation='tanh',
            recurrent_initializer = tf.keras.initializers.GlorotUniform(seed=random_seed),
            kernel_initializer = tf.keras.initializers.GlorotUniform(seed=random_seed) ,
            return_sequences=False)(x)
    
    
    x = bln_layer(stateful = True, batchsize= batch_size, name = 'bn2', 
                  batch_moving_mean = b_mm, batch_moving_var = b_mv,
                  feature_moving_mean = f_mm, feature_moving_var = f_mv)(x) 
    
    x = dense_layer(units = 32, name = 'dense1', random_seed=random_seed)(x)
    x = tf.keras.layers.Activation('tanh')(x)
    x = bln_layer(stateful = True, batchsize= batch_size, name = 'bn3', 
                  batch_moving_mean = b_mm, batch_moving_var = b_mv,
                  feature_moving_mean = f_mm, feature_moving_var = f_mv)(x)
           
    
    output_lyr = dense_layer(units = dense_units, name = 'dense2', random_seed=random_seed)(x)    
    return tf.keras.Model(inputs = [input_lyr], outputs = [output_lyr])

In [10]:
model_bln_layer = BLNLayer_model(inputshape = (80), max_features = max_features, 
                                        embed_size = 512, random_seed = random_seed,
                                        lstm_unit = 128 , dense_units= num_classes,
                                        batch_size = minibatch,
                                        b_mm = False, b_mv = False,
                                        f_mm = False, f_mv = False
                                        )



In [11]:
model_bln_layer.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(25, 80)]                0         
_________________________________________________________________
embedding (Embedding)        (25, 80, 512)             10240000  
_________________________________________________________________
lstm (LSTM)                  (25, 80, 128)             328192    
_________________________________________________________________
bn1 (bln_layer)              (25, 80, 128)             24738     
_________________________________________________________________
lstm_1 (LSTM)                (25, 64)                  49408     
_________________________________________________________________
bn2 (bln_layer)              (25, 64)                  308       
_________________________________________________________________
dense1 (dense_layer)         (25, 32)                  2080  

# callbacks

In [12]:
# Callbacks for saving best model and tensorboard
folder_name = '/rnn/bln_layer_imdb_batchSize_' + str(minibatch)
 
save_bm_cb = save_best_model_callback(folder_name)
tb_cb = tensorboard_callback(folder_name)

# Callback for resetting moving mean and variances at the end of each epoch
bln_layer_cb = bln_callback()

bln_layer_cb_list = [save_bm_cb, tb_cb, bln_layer_cb]

In [13]:
adam = tf.keras.optimizers.Adam(learning_rate= learning_rate, beta_1=.9, beta_2=.999, epsilon=1e-08)

model_bln_layer.compile(optimizer = adam,
                               loss = tf.keras.losses.BinaryCrossentropy(from_logits=True),
                               metrics = [tf.keras.metrics.BinaryAccuracy()])

In [14]:
model_bln_layer_history =  model_bln_layer.fit(train_dataset.take(number_batches_train), epochs=epochs,
                                                verbose=1, callbacks=bln_layer_cb_list,
                                                validation_data=valid_dataset.take(number_batches_valid),
                                                shuffle=True)

Train for 160 steps, validate for 40 steps
Epoch 1/10
160/160 [==============================] - 141s 884ms/step - loss: 0.5864 - binary_accuracy: 0.6620 - val_loss: 0.5051 - val_binary_accuracy: 0.7690
Epoch 2/10
160/160 [==============================] - 143s 894ms/step - loss: 0.3282 - binary_accuracy: 0.8585 - val_loss: 0.6083 - val_binary_accuracy: 0.7660
Epoch 3/10
160/160 [==============================] - 146s 915ms/step - loss: 0.1776 - binary_accuracy: 0.9358 - val_loss: 0.8178 - val_binary_accuracy: 0.7220
Epoch 4/10
160/160 [==============================] - 145s 904ms/step - loss: 0.0986 - binary_accuracy: 0.9685 - val_loss: 0.9108 - val_binary_accuracy: 0.7460
Epoch 5/10
160/160 [==============================] - 146s 912ms/step - loss: 0.0523 - binary_accuracy: 0.9833 - val_loss: 1.1147 - val_binary_accuracy: 0.7450
Epoch 6/10
160/160 [==============================] - 145s 907ms/step - loss: 0.0424 - binary_accuracy: 0.9850 - val_loss: 1.1855 - val_binary_accuracy: 0.75

In [15]:
del model_bln_layer, save_bm_cb, tb_cb, bln_layer_cb, bln_layer_cb_list
reset_graph()

session is clear


In [16]:
# Select the model with the lowest validation loss
weights_path = './models/rnn/bln_layer_imdb_batchSize_25/best_model.h5'

In [17]:
save_eval_path = "./logs/" + folder_name + '/'+ str(number_batches_train) +"_sorted_evaluation.pkl"
evaluation = grid_serach(BLNLayer_model, test_dataset,
                         batch_size = minibatch, sort=True,
                         save_eval_path = save_eval_path,
                         weights_path = weights_path,
                         loss = tf.keras.losses.BinaryCrossentropy(from_logits=True),
                         metrics = tf.keras.metrics.BinaryAccuracy())



1000/1000 [==============================] - 315s 315ms/step - loss: 0.6630 - binary_accuracy: 0.7690
{'Bmm_True Bmv_True Fmm_True Fmv_True': [0.6629747842252255, 0.769]}
session is clear
1000/1000 [==============================] - 316s 316ms/step - loss: 0.6498 - binary_accuracy: 0.7689
{'Bmm_True Bmv_True Fmm_True Fmv_True': [0.6629747842252255, 0.769], 'Bmm_True Bmv_True Fmm_True Fmv_False': [0.6497624014168978, 0.76892]}
session is clear
1000/1000 [==============================] - 316s 316ms/step - loss: 0.6629 - binary_accuracy: 0.7690
{'Bmm_True Bmv_True Fmm_True Fmv_True': [0.6629747842252255, 0.769], 'Bmm_True Bmv_True Fmm_True Fmv_False': [0.6497624014168978, 0.76892], 'Bmm_True Bmv_True Fmm_False Fmv_True': [0.6629162631332874, 0.76896]}
session is clear
1000/1000 [==============================] - 317s 317ms/step - loss: 0.6497 - binary_accuracy: 0.7688
{'Bmm_True Bmv_True Fmm_True Fmv_True': [0.6629747842252255, 0.769], 'Bmm_True Bmv_True Fmm_True Fmv_False': [0.649762401

1000/1000 [==============================] - 314s 314ms/step - loss: 0.4963 - binary_accuracy: 0.7708
{'Bmm_True Bmv_True Fmm_True Fmv_True': [0.6629747842252255, 0.769], 'Bmm_True Bmv_True Fmm_True Fmv_False': [0.6497624014168978, 0.76892], 'Bmm_True Bmv_True Fmm_False Fmv_True': [0.6629162631332874, 0.76896], 'Bmm_True Bmv_True Fmm_False Fmv_False': [0.6497238198667765, 0.76884], 'Bmm_True Bmv_False Fmm_True Fmv_True': [0.4987801894545555, 0.7726], 'Bmm_True Bmv_False Fmm_True Fmv_False': [0.4999744267165661, 0.77228], 'Bmm_True Bmv_False Fmm_False Fmv_True': [0.49874343141913413, 0.77256], 'Bmm_True Bmv_False Fmm_False Fmv_False': [0.49993953239917754, 0.77232], 'Bmm_False Bmv_True Fmm_True Fmv_True': [0.6067768514379859, 0.7694], 'Bmm_False Bmv_True Fmm_True Fmv_False': [0.5972367646098137, 0.76944], 'Bmm_False Bmv_True Fmm_False Fmv_True': [0.6067867462709546, 0.7694], 'Bmm_False Bmv_True Fmm_False Fmv_False': [0.5972526565939188, 0.7694], 'Bmm_False Bmv_False Fmm_True Fmv_True': 

In [18]:
evaluation

[('Bmm_False Bmv_False Fmm_True Fmv_False', [0.49630365321040154, 0.77116]),
 ('Bmm_False Bmv_False Fmm_False Fmv_False', [0.4963075534701347, 0.77116]),
 ('Bmm_False Bmv_False Fmm_True Fmv_True', [0.4963140346705914, 0.77092]),
 ('Bmm_False Bmv_False Fmm_False Fmv_True', [0.4963227770626545, 0.77084]),
 ('Bmm_True Bmv_False Fmm_False Fmv_True', [0.49874343141913413, 0.77256]),
 ('Bmm_True Bmv_False Fmm_True Fmv_True', [0.4987801894545555, 0.7726]),
 ('Bmm_True Bmv_False Fmm_False Fmv_False', [0.49993953239917754, 0.77232]),
 ('Bmm_True Bmv_False Fmm_True Fmv_False', [0.4999744267165661, 0.77228]),
 ('Bmm_False Bmv_True Fmm_True Fmv_False', [0.5972367646098137, 0.76944]),
 ('Bmm_False Bmv_True Fmm_False Fmv_False', [0.5972526565939188, 0.7694]),
 ('Bmm_False Bmv_True Fmm_True Fmv_True', [0.6067768514379859, 0.7694]),
 ('Bmm_False Bmv_True Fmm_False Fmv_True', [0.6067867462709546, 0.7694]),
 ('Bmm_True Bmv_True Fmm_False Fmv_False', [0.6497238198667765, 0.76884]),
 ('Bmm_True Bmv_True F

# 2.Using  Batch Normalization implemented in Keras

In [19]:
def bn_keras_model(inputshape = (80), max_features = 20000, embed_size = 512, random_seed = 100,
                          lstm_unit = 128 , dense_units= 1, batch_size = 60):
    

    
    input_lyr = tf.keras.Input(shape = inputshape, batch_size=batch_size, name = 'input')
    
    x = Embedding(max_features, embed_size,
                  embeddings_initializer=tf.keras.initializers.GlorotUniform(seed=random_seed))(input_lyr)
    
    x = LSTM(lstm_unit, dropout=0.2, recurrent_dropout=0.2, activation='tanh',
            recurrent_initializer = tf.keras.initializers.GlorotUniform(seed=random_seed),
            kernel_initializer = tf.keras.initializers.GlorotUniform(seed=random_seed) ,
            return_sequences=True)(x)
    
    x =  tf.keras.layers.BatchNormalization(momentum = 0.99,  name = 'bn1') (x) 
    
    x = LSTM(lstm_unit//2, dropout=0.2,  recurrent_dropout=0.2, activation='tanh',
            recurrent_initializer = tf.keras.initializers.GlorotUniform(seed=random_seed),
            kernel_initializer = tf.keras.initializers.GlorotUniform(seed=random_seed) ,
            return_sequences=False)(x)
    x =  tf.keras.layers.BatchNormalization(momentum = 0.99,  name = 'bn2') (x) 
    
    x = dense_layer(units = 32, name = 'dense1', random_seed=random_seed)(x)
    x = tf.keras.layers.Activation('tanh')(x)
    x =  tf.keras.layers.BatchNormalization(momentum = 0.99,  name = 'bn3') (x) 

    output_lyr = dense_layer(units = dense_units, name = 'dense2', random_seed=random_seed)(x)    
    return tf.keras.Model(inputs = [input_lyr], outputs = [output_lyr])


In [20]:
model_bn_keras = bn_keras_model(inputshape= (80), max_features = max_features, 
                                embed_size= 512, random_seed = random_seed,
                                lstm_unit = 128 , dense_units= num_classes,
                                batch_size = minibatch)
model_bn_keras.summary()

#### Compiling 
adam = tf.keras.optimizers.Adam(learning_rate= learning_rate, beta_1=.9, beta_2=.999, epsilon=1e-08)
model_bn_keras.compile(optimizer = adam,
                       loss = tf.keras.losses.BinaryCrossentropy(from_logits=True),
                       metrics = [tf.keras.metrics.BinaryAccuracy()])

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(25, 80)]                0         
_________________________________________________________________
embedding (Embedding)        (25, 80, 512)             10240000  
_________________________________________________________________
lstm (LSTM)                  (25, 80, 128)             328192    
_________________________________________________________________
bn1 (BatchNormalization)     (25, 80, 128)             512       
_________________________________________________________________
lstm_1 (LSTM)                (25, 64)                  49408     
_________________________________________________________________
bn2 (BatchNormalization)     (25, 64)                  256       
_________________________________________________________________
dense1 (dense_layer)         (25, 32)                  2080  

In [21]:
# Callbacks for saving best model and tensorboard
folder_name = '/rnn/bn_Keras_imdb_batchSize_' + str(minibatch)

save_bm_cb = save_best_model_callback(folder_name)
tb_cb = tensorboard_callback(folder_name)


bn_keras_cb_list = [save_bm_cb, tb_cb]

In [22]:
model_bn_keras_history =  model_bn_keras.fit(train_dataset.take(number_batches_train),
                                             epochs = epochs, verbose = 1, 
                                             callbacks = bn_keras_cb_list,
                                             validation_data = valid_dataset.take(number_batches_valid),
                                             shuffle = True)

Train for 160 steps, validate for 40 steps
Epoch 1/10
160/160 [==============================] - 144s 898ms/step - loss: 0.7140 - binary_accuracy: 0.5182 - val_loss: 0.6675 - val_binary_accuracy: 0.4890
Epoch 2/10
160/160 [==============================] - 144s 899ms/step - loss: 0.4985 - binary_accuracy: 0.7473 - val_loss: 0.6189 - val_binary_accuracy: 0.6950
Epoch 3/10
160/160 [==============================] - 144s 901ms/step - loss: 0.2498 - binary_accuracy: 0.9020 - val_loss: 0.7821 - val_binary_accuracy: 0.7340
Epoch 4/10
160/160 [==============================] - 143s 896ms/step - loss: 0.1636 - binary_accuracy: 0.9373 - val_loss: 0.9464 - val_binary_accuracy: 0.7070
Epoch 5/10
160/160 [==============================] - 143s 895ms/step - loss: 0.1099 - binary_accuracy: 0.9628 - val_loss: 0.9451 - val_binary_accuracy: 0.7230
Epoch 6/10
160/160 [==============================] - 144s 903ms/step - loss: 0.0646 - binary_accuracy: 0.9797 - val_loss: 1.2822 - val_binary_accuracy: 0.70

In [23]:
# Select the model with the lowest validation loss
weights_path = './models/rnn/bn_Keras_imdb_batchSize_25/best_model.h5'
model_bn_keras.load_weights(weights_path)

In [24]:
model_bn_keras_test_history = model_bn_keras.evaluate(test_dataset)

1000/1000 [==============================] - 316s 316ms/step - loss: 0.6203 - binary_accuracy: 0.6982


In [25]:
del model_bn_keras, save_bm_cb, tb_cb, bn_keras_cb_list 
reset_graph()

session is clear


# 3.Using  Layer normalization  implemented in Keras


In [26]:
def ln_keras_model(inputshape = (80), max_features = 20000, embed_size = 512, random_seed = 100,
                          lstm_unit = 128 , dense_units= 1, batch_size = 60):
    

    
    input_lyr = tf.keras.Input(shape = inputshape, batch_size=batch_size, name = 'input')
    
    x = Embedding(max_features, embed_size,
                  embeddings_initializer=tf.keras.initializers.GlorotUniform(seed=random_seed))(input_lyr)
    
    x = LSTM(lstm_unit, dropout=0.2, recurrent_dropout=0.2, activation='tanh',
            recurrent_initializer = tf.keras.initializers.GlorotUniform(seed=random_seed),
            kernel_initializer = tf.keras.initializers.GlorotUniform(seed=random_seed) ,
            return_sequences=True)(x)
    x =  tf.keras.layers.LayerNormalization()(x) 

    x = LSTM(lstm_unit//2, dropout=0.2,  recurrent_dropout=0.2, activation='tanh',
            recurrent_initializer = tf.keras.initializers.GlorotUniform(seed=random_seed),
            kernel_initializer = tf.keras.initializers.GlorotUniform(seed=random_seed) ,
            return_sequences=False)(x)
    x =  tf.keras.layers.LayerNormalization()(x) 
    
    x = dense_layer(units = 32, name = 'dense1', random_seed=random_seed)(x)
    x = tf.keras.layers.Activation('tanh')(x)
    x =  tf.keras.layers.LayerNormalization()(x) 

    
    output_lyr = dense_layer(units = dense_units, name = 'dense2', random_seed=random_seed)(x)    
    return tf.keras.Model(inputs = [input_lyr], outputs = [output_lyr])



In [27]:
model_ln_keras = ln_keras_model(inputshape= (80), max_features = max_features, 
                                embed_size= 512, random_seed = random_seed,
                                lstm_unit = 128 , dense_units= num_classes,
                                batch_size = minibatch)
model_ln_keras.summary()

#### Compiling 
adam = tf.keras.optimizers.Adam(learning_rate= learning_rate, beta_1=.9, beta_2=.999, epsilon=1e-08)

model_ln_keras.compile(optimizer = adam,
                       loss = tf.keras.losses.BinaryCrossentropy(from_logits=True),
                       metrics = [tf.keras.metrics.BinaryAccuracy()])

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(25, 80)]                0         
_________________________________________________________________
embedding (Embedding)        (25, 80, 512)             10240000  
_________________________________________________________________
lstm (LSTM)                  (25, 80, 128)             328192    
_________________________________________________________________
layer_normalization (LayerNo (25, 80, 128)             256       
_________________________________________________________________
lstm_1 (LSTM)                (25, 64)                  49408     
_________________________________________________________________
layer_normalization_1 (Layer (25, 64)                  128       
_________________________________________________________________
dense1 (dense_layer)         (25, 32)                  2080  

In [28]:
# Callbacks for saving best model and tensorboard
folder_name = '/rnn/ln_Keras_imdb_batchSize_' + str(minibatch) 

save_bm_cb = save_best_model_callback(folder_name)
tb_cb = tensorboard_callback(folder_name)

ln_keras_cb_list = [save_bm_cb, tb_cb]

In [29]:
model_ln_keras_history =  model_ln_keras.fit(train_dataset.take(number_batches_train),
                                             epochs=epochs, verbose=1, 
                                             callbacks=ln_keras_cb_list,
                                             validation_data=valid_dataset.take(number_batches_valid),
                                             shuffle=True)

Train for 160 steps, validate for 40 steps
Epoch 1/10
160/160 [==============================] - 144s 898ms/step - loss: 0.7015 - binary_accuracy: 0.4997 - val_loss: 0.7034 - val_binary_accuracy: 0.4880
Epoch 2/10
160/160 [==============================] - 143s 893ms/step - loss: 0.5430 - binary_accuracy: 0.7085 - val_loss: 0.6060 - val_binary_accuracy: 0.7330
Epoch 3/10
160/160 [==============================] - 144s 898ms/step - loss: 0.2694 - binary_accuracy: 0.8888 - val_loss: 0.8571 - val_binary_accuracy: 0.6910
Epoch 4/10
160/160 [==============================] - 142s 890ms/step - loss: 0.1182 - binary_accuracy: 0.9580 - val_loss: 0.9089 - val_binary_accuracy: 0.7580
Epoch 5/10
160/160 [==============================] - 143s 894ms/step - loss: 0.0643 - binary_accuracy: 0.9790 - val_loss: 0.9540 - val_binary_accuracy: 0.7570
Epoch 6/10
160/160 [==============================] - 143s 894ms/step - loss: 0.0286 - binary_accuracy: 0.9925 - val_loss: 1.0657 - val_binary_accuracy: 0.75

In [30]:
# Select the model with the lowest validation loss

weights_path = './models/rnn/ln_Keras_imdb_batchSize_25/best_model.h5'

model_ln_keras.load_weights(weights_path)

In [31]:
model_ln_keras_test_history = model_ln_keras.evaluate(test_dataset)

1000/1000 [==============================] - 312s 312ms/step - loss: 0.6386 - binary_accuracy: 0.7105


In [32]:
del model_ln_keras,save_bm_cb, ln_keras_cb_list
reset_graph()

session is clear


# 4.writing pickle files

In [33]:
write_pickle_file('./logs/rnn/ln_Keras_imdb_batchSize_25/model_ln_keras_history_25.pickle', model_ln_keras_history.history)
write_pickle_file('./logs/rnn/ln_Keras_imdb_batchSize_25/model_ln_keras_test_history_25.pickle', model_ln_keras_test_history)

write_pickle_file('./logs/rnn/bn_Keras_imdb_batchSize_25/model_bn_keras_history_25.pickle', model_bn_keras_history.history)
write_pickle_file('./logs/rnn/bn_Keras_imdb_batchSize_25/model_bn_keras_test_history_25.pickle', model_bn_keras_test_history)

write_pickle_file('./logs/rnn/bln_layer_imdb_batchSize_25/model_bln_keras_history_25.pickle', model_bln_layer_history.history)
write_pickle_file('./logs/rnn/bln_layer_imdb_batchSize_25/model_bln_keras_test_history_25.pickle', evaluation)